In [1]:
import numpy as np
import pandas as pd
import math
import cv2

In [3]:
#Working Directory
import os
#For Train Images
os.chdir("/Users/pratham/Desktop/ML:DL Practice/Age Detection/train_DETg9GD/Train")

In [4]:
#load all the train images in a list
relevant_path = "/Users/pratham/Desktop/ML:DL Practice/Age Detection/train_DETg9GD/Train"
included_extensions = ['jpg', 'bmp', 'png', 'gif']
file_names = [fn for fn in os.listdir(relevant_path)
              if any(fn.endswith(ext) for ext in included_extensions)]

In [7]:
#HOG Descriptor
winSize = (28,28)
blockSize = (8,8)
blockStride = (4,4)
cellSize = (8,8)
nbins = 9
derivAperture = 1
winSigma = -1.
histogramNormType = 0
L2HysThreshold = 0.2
gammaCorrection = 1
nlevels = 64
signedGradient = True
 
hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,
      derivAperture,winSigma,histogramNormType,L2HysThreshold,
      gammaCorrection,nlevels, signedGradient)

In [8]:
image_train = []
for i in range(len(file_names)):
    image_train.append(cv2.imread(file_names[i],0))

In [9]:
#compute(img[, winStride[, padding[, locations]]]) -> descriptors
winStride = (8,8)
padding = (8,8)
locations = ((10,20),)

In [11]:
hist = []
for i in range(len(image_train)):
    hist.append(hog.compute(image_train[i],winStride,padding,locations))

In [15]:
hist = np.squeeze(hist)

In [16]:
hist[0].shape

(324,)

In [49]:
#For test data
os.chdir("/Users/pratham/Desktop/ML:DL Practice/Age Detection/test_Bh8pGW3")
test = pd.read_csv("Test.csv")

In [17]:
#Test
os.chdir("/Users/pratham/Desktop/ML:DL Practice/Age Detection/test_Bh8pGW3/Test")

In [18]:
#load all the test images in a list
relevant_path = "/Users/pratham/Desktop/ML:DL Practice/Age Detection/test_Bh8pGW3/Test"
included_extensions = ['jpg', 'bmp', 'png', 'gif']
file_names = [fn for fn in os.listdir(relevant_path)
              if any(fn.endswith(ext) for ext in included_extensions)]

In [19]:
image_test = []
for i in range(len(file_names)):
    image_test.append(cv2.imread(file_names[i],0))

In [20]:
hist_test = []
for i in range(len(image_test)):
    hist_test.append(hog.compute(image_test[i],winStride,padding,locations))

In [21]:
hist_test = np.squeeze(hist_test)

In [24]:
#for train data
os.chdir("/Users/pratham/Desktop/ML:DL Practice/Age Detection/train_DETg9GD")
train = pd.read_csv("Train.csv")

In [25]:
train_label = train['Class']

In [28]:
#Label Encoding
train_label = train_label.map({'YOUNG': 0, 'MIDDLE': 1, 'OLD': 2})

In [31]:
train_label = np.array(train_label)

In [ ]:
#Model

In [22]:
def svmInit(C=12.5, gamma=0.50625):
  model = cv2.ml.SVM_create()
  model.setGamma(gamma)
  model.setC(C)
  model.setKernel(cv2.ml.SVM_RBF)
  model.setType(cv2.ml.SVM_C_SVC)
  
  return model

In [23]:
def svmTrain(model, samples, responses):
  model.train(samples, cv2.ml.ROW_SAMPLE, responses)
  return model

def svmPredict(model, samples):
  return model.predict(samples)[1].ravel()

In [32]:
model = svmInit()
svmTrain(model, hist, train_label)

<ml_SVM 0x104e3f470>

In [34]:
y_pred = svmPredict(model,hist_test)

In [35]:
type(y_pred)

numpy.ndarray

In [36]:
Y_pred = pd.DataFrame(y_pred)

In [44]:
Y_pred.groupby(Y_pred.iloc[0,]).count()

,0
0,
2.0,1


In [45]:
y_pred
dic = {0: 'YOUNG', 1: 'MIDDLE', 2: 'OLD'}

In [47]:
#Convert test predictions into true categories
test_cls = [dic[x] for x in y_pred]
test_cls[:4]

['OLD', 'OLD', 'OLD', 'OLD']

In [50]:
sub = pd.DataFrame({
    "Class": test_cls,
    "ID": test['ID']
})

In [51]:
sub.to_csv("submissionHOG.csv", index=False)